<a href="https://colab.research.google.com/github/lonestarchaser/SED/blob/main/Step_1_Polygon_Search_Specific_IRSA_Catalogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's just go to IRSA Catalog Search Tool and pull the polygon points

Some acronyms you might like to know:
* ADQL = Astronomical Data Query Language
* TAP = Table Access Protocol


## First, install Astropy libraries and import packages and subpackages

These aren't normally included in Colab. You can save yourself some time and only have to run this once per time you open this.

In [5]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery gala

## Import subpackages


In [6]:
from google.colab import drive
import requests
from astroquery.ipac.irsa import Irsa

## Connect to your Google Drive

In order to save the file to Google Drive, you need to mount you Google Drive to this Google Colab notebook. This will allow you to access your Google Drive files directly from your notebook.

It's going to ask for a lot of permissions.  Just get over it.

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


Now you can list the files in that directory.

You can do this to confirm the file is actually there:

``!ls "/content/drive/My Drive/Colab Notebooks" ``

# Connect to the TAP Server and run a query

I took the code from here:  https://irsa.ipac.caltech.edu/docs/program_interface/TAP.html

Then, in an act of desperation, I got ChatGPT to fix it for me.

I just said, "convert this into something I can use in google colab in python:"

`` https://irsa.ipac.caltech.edu/TAP/sync?QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT(ra,dec),POLYGON(209.80225,53.34894,209.80225,55.34894,211.80225,54.34894))=1  ``

Then I did a ton of extra stuff to it.

In [8]:
def run_my_fancy_query(mySourceToUse,betterFileName,queryDetails,locationTag):

    url = 'https://irsa.ipac.caltech.edu/TAP/sync'

    query = (
        f"SELECT '{mySourceToUse}' AS Source, g.* "
        f"FROM {mySourceToUse} AS g "
        f"WHERE CONTAINS(POINT(g.ra, g.dec), {queryDetails} "
    )

    funData = {
        'FORMAT': 'IPAC_TABLE',
        'QUERY': query
    }

    response = requests.post(url, data=funData)

    # Print the response object
    print(response)

    if response.status_code == 200:
        response_content = response.content
        output_file_path = "/content/drive/My Drive/Colab Notebooks/" + betterFileName + locationTag + ".tbl"
        with open(output_file_path, 'wb') as f:
            f.write(response.content)
        print("File " + betterFileName + locationTag + ".tbl" + " created successfully.")
    else:
        # Handle the error condition
        print(f"File " + betterFileName + locationTag + ".tbl" + " not created!")
        print(f"Error: {response.status_code}")

    #print(response_content)

# Just pull the IRSA Catalogs I want to use

This could use an improvement where it handles 502 errors, but that's not happening now that I'm just reaching out for specific catalogs that I alraedy know work.




In [10]:
IrsaCatalogsToUse = ['slphotdr4',      'unwise_2019', 'fp_psc',       'akari_irc', 'allwise_p3as_psd', 'catwise_2020', 'akari_fis', 'gaia_dr3_source']
BetterNamesToUse =  ['SEIP_slphotdr4', 'unwise_2019', '2mass_fp_psc', 'akari_irc', 'allwise_p3as_psd', 'catwise_2020', 'akari_fis', 'gaia_dr3_source']

queryDetails = "POLYGON(209, 55, 207, 55, 207, 50, 209, 50)) = 1"
locationTag = "_MyFunPolygon"

for index, row in enumerate(IrsaCatalogsToUse):
    if index >= 0:# and index < 895:
        print('---------------------------------------------------------------------------')
        #print(index, "-", row)
        mySourceToUse = str(row)

        betterFileName = BetterNamesToUse[index]
        print("Using catalog: " + mySourceToUse + " and saving as:", betterFileName + locationTag + ".tbl ...")

        run_my_fancy_query(mySourceToUse,betterFileName,queryDetails,locationTag)

print("Done with all catalogs!")

---------------------------------------------------------------------------
Using catalog: slphotdr4 and saving as: SEIP_slphotdr4_MyFunPolygon.tbl ...
<Response [200]>
File SEIP_slphotdr4_MyFunPolygon.tbl created successfully.
---------------------------------------------------------------------------
Using catalog: unwise_2019 and saving as: unwise_2019_MyFunPolygon.tbl ...
<Response [200]>
File unwise_2019_MyFunPolygon.tbl created successfully.
---------------------------------------------------------------------------
Using catalog: fp_psc and saving as: 2mass_fp_psc_MyFunPolygon.tbl ...
<Response [200]>
File 2mass_fp_psc_MyFunPolygon.tbl created successfully.
---------------------------------------------------------------------------
Using catalog: akari_irc and saving as: akari_irc_MyFunPolygon.tbl ...
<Response [200]>
File akari_irc_MyFunPolygon.tbl created successfully.
---------------------------------------------------------------------------
Using catalog: allwise_p3as_psd a

# For Troubleshooting

If we're just running one at a time

In [9]:
# Do this if it's just one at a time
mySourceToUse = 'fp_psc'
betterFileName = '2mass_fp_psc'
queryDetails = "POLYGON(209, 55, 207, 55, 207, 50, 209, 50)) = 1"
locationTag = "_MyFunPolygon"


print("    Use this one:", mySourceToUse)

run_my_fancy_query(mySourceToUse,betterFileName,queryDetails,locationTag)


print("done!")

    Use this one: fp_psc
<Response [200]>
File 2mass_fp_psc_MyFunPolygon.tbl created successfully.
done!
